In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


When creating new attributes, it is important to decide which MOI instance should handle setting and/or getting it.

The typical MOI structure is as follows:
* Caching optimizer
  * Cache
  * Bridge optimizer
    * Bridges
    * Optimizer

We can classify attributes in different categories. They can be distinguished by their super type, either `AbstractOptimizerAttribute` or other (i.e. `AbstractModelAttribute`, `AbstractConstraintAttribute` or `AbstractVariableAttribute`) and by the traits `is_set_by_optimize` (`false` by default) and `is_copyable` (`!is_set_by_optimize` by default).

The table below shows an example of attribute in each category (except the last column for which no example exists).
Columns names are the value of `is_set_by_optimize`/`is_copyable` for the attribute.

|                     |   `false`/`false`   |  `false`/`true`  |  `true`/`false`  | `true`/`true` |
|:-------------------:|:-------------------:|:----------------:|:----------------:|:-------------:|
| optimizer attribute |        `SolverName` |  `VerboseLevel`  |                  |               |
| other               | `NumberOfVariables` | `ObjectiveSense` | `VariablePrimal` |               |

The attributes in the first column should both implement `is_set_by_optimize` and `is_copyable`,
the attributes in the second column can keep the default methods and the attributes in the
third columns only need to implement `is_set_by_optimize`.

In [2]:
using JuMP
struct TimeLimit <: MOI.AbstractOptimizerAttribute end
attr = TimeLimit()
MOI.is_set_by_optimize(attr)

false

In [3]:
MOI.is_copyable(attr)

true

As we can see above, the attribute is in the secon column by default.

## Attributes and caching optimizer

The caching optimizer works as follows with attributes:
* For `set`:
  * If it is an optimizer attribute, it is set both to the cache and optimizer;
  * otherwise it is set to the cache and
    * if the state is `ATTACHED_OPTIMIZER`, it is set to the optimizer too, if the optimizer throws `NotAllowedError`, it is detached.

    The attribute will be copied before `optimize!` if the state is not `ATTACHED_OPTIMIZER`.
    Note that names are an exception, they are only set to the cache an never to the optimizer.
    This allows optimizer not supporting names to be used in a model using names.
* For `get`:
  * If it is an optimizer attribute or it `is_set_by_optimize`, it is got from the optimizer;
  * otherwise it is got from the cache.

In [4]:
using MosekTools
cached = MOIU.CachingOptimizer(MOIU.UniversalFallback(JuMP._MOIModel{Float64}()), MOIU.AUTOMATIC)
optimizer = Mosek.Optimizer(QUIET=true)
MOIU.reset_optimizer(cached, optimizer)
MOIU.state(cached)

┌ Info: Precompiling MosekTools [1ec41992-ff65-5c91-ac43-2df89e9693a4]
└ @ Base loading.jl:1186


EMPTY_OPTIMIZER::CachingOptimizerState = 1

The state is empty optimizer so modifications to the model will not be copied to the model until it is attached.
This is not the case of optimizer attributes.

In [5]:
struct TimeLimit <: MOI.AbstractOptimizerAttribute end

In [6]:
MOI.set(cached, TimeLimit(), 0.5)

MathOptInterface.UnsupportedAttribute{TimeLimit}: MathOptInterface.UnsupportedAttribute{TimeLimit}: Attribute TimeLimit() is not supported by the model.

As we can see, the optimizer attribute is directly set to the optimizer even if the state is empty.
We can now implement it for Mosek.

In [7]:
MOI.supports(::MosekTools.MosekModel, ::TimeLimit) = true
function MOI.set(m::MosekTools.MosekModel, ::TimeLimit, t)
    MOI.set(m, MosekTools.DoubleParameter("MSK_DPAR_OPTIMIZER_MAX_TIME"), t)
end
function MOI.get(m::MosekTools.MosekModel, ::TimeLimit)
    MOI.get(m, MosekTools.DoubleParameter("MSK_DPAR_OPTIMIZER_MAX_TIME"))
end

In [8]:
MOI.set(cached, TimeLimit(), 0.5)
MOI.get(optimizer, TimeLimit())

0.5

When another optimizer is attached, the optimizer attribtes get copied directly.
This is because the optimizer is responsible for the `get` operation so it should always have the correct values
of the optimizer attributes even in the empty state.

In [9]:
import CSDP
MOIU.reset_optimizer(cached, CSDP.Optimizer())

MathOptInterface.UnsupportedAttribute{TimeLimit}: MathOptInterface.UnsupportedAttribute{TimeLimit}: Attribute TimeLimit() is not supported by the model.

In [10]:
MOIU.state(cached)

EMPTY_OPTIMIZER::CachingOptimizerState = 1

In [11]:
MOIU.drop_optimizer(cached)

When no optimizer is attached, `set` only stores optimizer attributes in the cache.

In [13]:
MOI.set(cached, TimeLimit(), 1.0)

1.0

Note that optimizer attributes are not cleared by `empty!`.

In [14]:
MOI.empty!(optimizer)
MOI.get(optimizer, TimeLimit())

0.5

In [15]:
MOIU.reset_optimizer(cached, optimizer)
MOI.get(optimizer, TimeLimit())

1.0

## Attributes and bridges

All attributes `get`/`set` calls are passed directly to the optimizer except for `AbstractConstraintAttribute` of constraints that are bridged.
For these, the call is passed to the bridge which typically then use calls with the same attribute for the transformed constraints.

In [16]:
using MosekTools
optimizer = Mosek.Optimizer(QUIET=true)
bridged = MOI.Bridges.full_bridge_optimizer(optimizer, Float64)
model = direct_model(bridged)
@variable(model, x)
using LinearAlgebra
cref = @constraint(model, Symmetric([1 x; x 1]) in PSDCone())

[1  x;
 x  1] ∈ PSDCone()

In [17]:
MOI.Bridges.is_bridged(backend(model), typeof(index(cref)))

true

In [18]:
typeof(MOI.Bridges.bridge(backend(model), index(cref)))

MathOptInterface.Bridges.VectorSlackBridge{Float64,MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.PositiveSemidefiniteConeTriangle}

In [19]:
optimize!(model)

In [20]:
struct UnknownConstraintAttribute <: MOI.AbstractConstraintAttribute end
MOI.is_set_by_optimize(::UnknownConstraintAttribute) = true

In [21]:
MOI.get(model, UnknownConstraintAttribute(), cref)

ArgumentError: ArgumentError: Constraint bridge of type `MathOptInterface.Bridges.VectorSlackBridge{Float64,MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.PositiveSemidefiniteConeTriangle}` does not support accessing the attribute `UnknownConstraintAttribute()`.